In [1]:
from datetime import *
import smtplib
from email.mime.text import MIMEText

class library:
  def __init__(self):
    pass
  added_books = {}
  borrowed_books = {}
  reserved_books = {}
  user_emails = {}

In [7]:
class management(library):
  def __init__(self):
    pass
  def add(self,book_name,copies=1):
    if (book_name not in super().added_books):
      super().added_books.update({book_name:copies})
      # If the library never had this book -> add it
      super().borrowed_books.update({book_name:[0]})
      # add an element in borrowed_books with the name of this book and make borrowed copies of it 0
      # number of borrowed copies is in a list so this list would have more info about borrowed book
      # so the format of borrowed_books would be:
      # dict(borrowed_books) = {book_name:[borrowed_copies,date_of_borrowing,user_that_borrowed]}
    else:
      super().added_books[book_name]+=copies
      # If the library once had this book -> increase copies

  def remove(self,book_name):
    if (book_name in super().added_books):
      super().added_books.pop(book_name)
    if (book_name in super().borrowed_books):
      super().borrowed_books.pop(book_name)
    # check if the book is existed in the library and remove it from library and borrowed books

  def display_books(self):
    # print(super().added_books)
    for x in super().added_books:
      print(f"{x} book has {super().added_books.get(x)} copies ")

  def display_borrowed_books(self):
    # print(super().borrowed_books)
    for x in super().borrowed_books:
      if (super().borrowed_books.get(x)[0] > 0):
        print(f"{super().borrowed_books.get(x)} copies of {x} book have been borrowed ")

  def send_email(self,name,receiver_email,book_name):
    # Configuration
    port = 587
    smtp_server = "smtp-relay.brevo.com"
    login = "7a8d24001@smtp-brevo.com"
    password = "RFLYaOC9KEbxtAJ0"
    sender_email = "yaqoubtest0@gmail.com"
    # receiver_email = "yaqoubfake0@gmail.com"
    # Plain text content
    text = f"Hi {name}, just wanted to notice you that the book you reserved ({book_name}) is available to borrow at the library. \nThanks. \nNaffa'a Library"

    # Create MIMEText object
    message = MIMEText(text, "plain")
    message["Subject"] = "About your reservation of book."
    message["From"] = sender_email
    message["To"] = receiver_email

    # Send the email
    with smtplib.SMTP(smtp_server, port) as server:
      server.starttls()  # Secure the connection
      server.login(login, password)
      server.sendmail(sender_email, receiver_email, message.as_string())


In [8]:
class users(management):
  def __init__(self,name,role,email):
    self.name = name
    # self.name = self.name.capitalize()
    self.role = role
    self.role = self.role.lower()
    self.email = email
    super().user_emails.update({name:email})

  def add(self,book_name,copies=1):
    if (self.role == "admin"):
      super().add(book_name,copies)
    else:
      print("Sorry, you are not authorized to make this instruction.")

  def remove(self,book_name):
    if (self.role == "admin"):
      super().remove(book_name)
    else:
      print("Sorry, you are not authorized to make this instruction.")

  borrowed_by_user = {}
  transaction_log = {}
  def borrow(self,book_name):
    if (book_name in super().added_books):
      # if the library have this book (either if it's borrowed or not)
      if(super().added_books[book_name] - super().borrowed_books[book_name][0] > 0):
        # if the library still have at least a copy to borrow it.
        super().borrowed_books[book_name][0]+=1
        # increase number of borrowed copies of that book
        '''
        super().borrowed_books[book_name].extend([datetime.now().date(),self.name])
        # modify borrowed_books dictionary format to be:
        # dict(borrowed_books) = {book_name:[borrowed_copies,date_of_borrowing,user_that_borrowed]}
        '''
        self.borrowed_by_user.update({book_name:datetime.now().date()})
        # add to borrowed_by_user dict a book_name that user borrowed and date of borrowing
        if(str(datetime.now().date()) in self.transaction_log):
          self.transaction_log[str(datetime.now().date())].extend([(self.name,'borrowed',book_name)])
        else:
          self.transaction_log.update({str(datetime.now().date()):[(self.name,'borrowed',book_name)]})
        # for transaction log if the day of the transaction is in log
        # then add the transaction to the date
        # if not, add the date and the transaction to it
      else:
        print(f"We're sorry all copies of {book_name} are borrowed at the moment.")
        answer = input("Would you like to reserve it when it's available?")
        yes = ["yes","Yes"]
        if (answer in yes):
          self.reserve(book_name)
    else:
      print("We're sorry the library hadn't get this book yet.")

  def display_user_details(self):
    if (len(self.borrowed_by_user) > 0):
      print(f"Hi {self.name} you borrowed:")
      for x in self.borrowed_by_user:
        borrowing_days = (datetime.now().date() - self.borrowed_by_user.get(x)).days
        if (borrowing_days > 7):
          print(f"{x} at {self.borrowed_by_user.get(x)} and the fines of borrowing it is: {borrowing_days-7} units.")
        else:
          print(f"{x} at {self.borrowed_by_user.get(x)} and there's no fines yet.")
    else:
      print("You didn't borrow any book yet")
    # rules are every borrowed book should be returned in a week.

  def return_book(self,book_name):
    if (book_name in super().borrowed_books and super().borrowed_books[book_name][0]>0):
      super().borrowed_books[book_name][0]-=1
      borrowing_days = (datetime.now().date() - self.borrowed_by_user.get(book_name)).days
      if (borrowing_days>7):
        print(f"You have to pay {borrowing_days-7} units fines because of overdue.")
      if(str(datetime.now().date()) in self.transaction_log):
        self.transaction_log[str(datetime.now().date())].extend([(self.name, 'returned',book_name)])
      else:
        self.transaction_log.update({str(datetime.now().date()):[(self.name,'returned',book_name)]})
      print("Thanks for being honest and returning the book :)")
      self.borrowed_by_user.pop(book_name)
      if (book_name in super().reserved_books):
        reserver = super().reserved_books[book_name][0]
        super().reserved_books[book_name].pop(0)
        reserver_email = super().user_emails[reserver]
        self.alert_reserver(reserver,reserver_email,book_name)

  def reserve(self,book_name):
    if (book_name in super().borrowed_books and super().borrowed_books[book_name][0]>0):
      if (book_name not in super().reserved_books):
        super().reserved_books.update({book_name:[self.name]})
      else:
        super().reserved_books[book_name].extend([self.name])
      print(f"You reserved {book_name} to borrow, whenever it's available we'll notify you by your email.")

  def alert_reserver(self,reserver,reserver_email,book_name):
    super().send_email(reserver,reserver_email,book_name)



In [11]:
yaqoub = users("Yaqoub", 'admin',"yaqoubfake0@gmail.com")
malek = users('Malek', 'regular',"malekcs04@gmail.com")
aktham = users('Aktham', 'regular',"akthamzayed04@gmail.com")
# yaqoub.add('ir', 2)
# yaqoub.add('calc', 4)
# yaqoub.remove('ir')
# yaqoub.remove('calc')
# malek.borrow('ir')
# malek.borrow('calc')
# ahmad.borrow('ir')
# yaqoub.borrow("ir")
# yaqoub.display_books()
# yaqoub.display_borrowed_books()
# malek.display_user_details()
